OrderBook Demo
-------------

Examples of placing orders and basic fill probability estimation

In [1]:
#Mosie Schrem
import pandas as pd
import numpy as np
import os
import math
import sys
sys.path.insert(0, os.path.join(os.getcwd(), 'code'))
import OrderBook as ob
import OrderUtil as ou
import FillProbabilitySimulator as fps

Examples of creating and processing orders:
--------

In [2]:
#load in intc OrderBook data
book = ob.OrderBook(message_filename='INTC_message.csv', orderbook_filename='INTC_OrderBook.csv')

Create an ask order at level 2, at time 38000, reevaluating order status at every BookUpdate 

In [3]:
order = ou.BookUpdatesOrder(orderbook=book, numupdates=1, timestamp=38000, level=2, is_buy=False)
print(order.get_opening_stats())

{'price': 273800, 'start_time': 38000, 'is_buy': False, 'orderstate': 'open', 'level': 2, 'start_index': 131939}


In [4]:
#process 100 updates to orderbook
for i in range(100):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38014.459199206, 'is_buy': False, 'orderstate': 'open', 'level': 2, 'end_index': 132039, 'price_delta': 0}


In [5]:
#process another 300 updates
for i in range(300):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38015.006473335, 'is_buy': False, 'orderstate': 'open', 'level': 2, 'end_index': 132339, 'price_delta': 0}


In [6]:
#process until executed or cancelled
while(order.order_type('open')):
    order.update()
    
#print stats at time when order was executed
print(order.get_closing_stats())

{'price': 273800, 'time': 38167.621597535, 'is_buy': False, 'orderstate': 'executed', 'level': 1, 'end_index': 134171, 'price_delta': 0}


We can also create orders by timestep of fixed size

Create an ask order at level 2, at time 38000, reevaluating order status every 0.5 seconds

In [7]:
order = ou.TimeOrder(orderbook=book, delta_t=0.5, timestamp=38000, level=2, is_buy=False)
print(order.get_opening_stats())

{'price': 273800, 'start_time': 38000, 'is_buy': False, 'orderstate': 'open', 'level': 2, 'start_index': 131939}


In [8]:
#process 50 seconds of data
for i in range(100):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38049.987429576, 'is_buy': False, 'orderstate': 'open', 'level': 3, 'end_index': 132820, 'price_delta': 0}


In [9]:
#process until executed or cancelled
while(order.order_type('open')):
    order.update()
#print stats at time when order was executed
print(order.get_closing_stats())

{'price': 273800, 'time': 38167.621597535, 'is_buy': False, 'orderstate': 'executed', 'level': 1, 'end_index': 134171, 'price_delta': 0}


Fill Probability Simulation Examples:
------

In the following example, we quote our buy and sell order on inside market, and reevaluate every BookUpdate

In [10]:
probability_simulator = fps.FillProbabilitySimulator(numupdates=1, order_tuple=(1 , 1),
                                                     orderbook=book, t_start=34200.01, t_end=50000)

probability_simulator.generate_matrix(num_samples=10000)

samples processed so far: 1000.0
samples processed so far: 2000.0
samples processed so far: 3000.0
samples processed so far: 4000.0
samples processed so far: 5000.0
samples processed so far: 6000.0
samples processed so far: 7000.0
samples processed so far: 8000.0
samples processed so far: 9000.0
samples processed so far: 10000.0


Display empirical fill probabilities for given midprice movement -1, 0, and 1

In [11]:
fill_matrix = probability_simulator.get_prob_matrix()
print(fill_matrix)

                   -1         0         1
none         0.552632  0.989007  0.425532
bid          0.447368  0.007060  0.000000
ask          0.000000  0.003933  0.574468
bid_and_ask  0.000000  0.000000  0.000000


Display frequencies of midprice movements in our sample

In [12]:
print(probability_simulator.get_quantities())

{-1: 38, 0: 9915, 1: 47}


In the following example, we quote our buy and sell order on inside market, and reevaluate every 100 BookUpdates

In [13]:
probability_simulator = fps.FillProbabilitySimulator(numupdates=100, order_tuple=(1 , 1),
                                                     orderbook=book, t_start=34200.01, t_end=50000)

probability_simulator.generate_matrix(num_samples=10000)

samples processed so far: 1000.0
samples processed so far: 2000.0
samples processed so far: 3000.0
samples processed so far: 4000.0
samples processed so far: 5000.0
samples processed so far: 6000.0
samples processed so far: 7000.0
samples processed so far: 8000.0
samples processed so far: 9000.0
samples processed so far: 10000.0


Display empirical fill probabilities for given midprice movement -1, 0, and 1

In [14]:
fill_matrix = probability_simulator.get_prob_matrix()
print(fill_matrix)

                   -1         0         1
none         0.344209  0.641853  0.342804
bid          0.587276  0.190920  0.009883
ask          0.020120  0.145216  0.626930
bid_and_ask  0.048396  0.022012  0.020383


Display frequencies of midprice movements in our sample

In [15]:
print(probability_simulator.get_quantities())

{-1: 1839, 0: 6542, 1: 1619}


In the next example we reevaluate our orders every second

In [16]:
probability_simulator = fps.FillProbabilitySimulator(timeunit=1, order_tuple=(1 , 1),
                                                     orderbook=book, t_start=34200.01, t_end=50000)
probability_simulator.generate_matrix(num_samples=10000)

samples processed so far: 1000.0
samples processed so far: 2000.0
samples processed so far: 3000.0
samples processed so far: 4000.0
samples processed so far: 5000.0
samples processed so far: 6000.0
samples processed so far: 7000.0
samples processed so far: 8000.0
samples processed so far: 9000.0
samples processed so far: 10000.0


Display empirical fill probabilities for given midprice movement -1, 0, and 1

In [17]:
fill_matrix = probability_simulator.get_prob_matrix()
print(fill_matrix)

                   -1         0         1
none         0.757576  0.893887  0.738693
bid          0.128788  0.059138  0.130653
ask          0.083333  0.046031  0.090452
bid_and_ask  0.030303  0.000944  0.040201


In [18]:
print(probability_simulator.get_quantities())

{-1: 264, 0: 9537, 1: 199}
